<a href="https://colab.research.google.com/github/ChristinaKou/Congitive-/blob/Dataset/CognvitivePlus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Initialisation and function definitions**

# **1. Initialisation and function definitions**

Import modules

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt 
import tarfile

from keras import layers


Using TensorFlow backend.


Paths

In [2]:
# Project directory
workingDir = '/Cognitive'
# Make a project folder if not existent
if not os.path.exists(workingDir):
    os.makedirs(workingDir)
os.chdir(workingDir) # Cd to this foder 
print("The current workig directory is",os.getcwd()[1:] )

The current workig directory is Cognitive


Functions

In [0]:
# List non hidden files in current folder
def nonHiddenfiles():
  fileList = [n_file for n_file in os.listdir() if not n_file.startswith('.')] 
  return fileList

# **2. Data Loading and preprocessing**



## 2.1 Download and decompress dataset

In [73]:
# Download dataset if not already downloaded
fileList = nonHiddenfiles()
if len(fileList)==0: # Check for non hidden files
  # Donwload 
  print("Downloading data ..")
  !wget -q --show-progress https://www.cs.toronto.edu/%7Ekriz/cifar-10-python.tar.gz
  fileList = nonHiddenfiles()
  print("Dataset", fileList[-1],  "downloaded" )
else:
  # Do not download
  print("Dataset", fileList[-1], "already downloaded" )
  fileList = nonHiddenfiles()

cifar-10-python.tar 100%[===================>] 162.60M  31.4MB/s    in 5.8s    
Dataset cifar-10-python.tar.gz downloaded


In [79]:
# Decompress dataset
fileList = nonHiddenfiles()
datasetFile = fileList[-1]
if datasetFile.endswith("tar.gz"):
  print("Decompressing dataset..")    
  tar = tarfile.open(datasetFile, "r:gz")    
  tar.extractall()
  tar.close()
  print("..into", tar.name)
  os.remove(tar.name) # delete compressed file
  fileList.remove(datasetFile)
else:
  print("Dataset uncompressed") 


Dataset uncompressed


## 2.2 Explore dataset

In [124]:
# Look into the folder
os.chdir(tar.members[0].name) 
# List non hidden content
contentList = nonHiddenfiles()
print(*contentList, sep = "\n")

data_batch_5
data_batch_2
data_batch_1
batches.meta
test_batch
readme.html
data_batch_4
data_batch_3


In [123]:
cd ..

/Cognitive
